In [1]:
import numpy as np
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import os
from PIL import Image
from skimage.measure import compare_ssim,compare_psnr
np.random.seed(73)

In [2]:
def compression(Data_in, code_in, stride_size, Q_scale, Q_type):
    compW = tf.constant(code_in)

    comp = tf.nn.conv2d(Data_in, compW, strides=[1, stride_size, stride_size, 1], padding='VALID')
    print(comp)

    compm = comp*Q_scale
    print(compm)

    compint = tf.cast(compm,Q_type)
    print(compint)
    
    return compint

In [3]:
block_size = 8
stride_size = 8
ncomp=4

sc_int=23.
Q_scale=1.

nrfs=10
Nx=2048
Ny=3840
NC = 1

f_code=h5py.File('./code_884_1x16_mix_4bit.h5','r')
code=f_code['data'][:]
code=code.astype(np.float32)
print("code shape: " + str(code.shape))

code shape: (8, 8, 1, 4)


In [4]:
with tf.name_scope('DATA'):
    Data_in = tf.placeholder(tf.float32,shape=[None,Nx,Ny,NC])
    print(Data_in)

Tensor("DATA/Placeholder:0", shape=(?, 2048, 3840, 1), dtype=float32)


In [5]:
comp_o=compression(Data_in, code, stride_size, Q_scale, tf.int8)
print(comp_o)

Tensor("Conv2D:0", shape=(?, 256, 480, 4), dtype=float32)
Tensor("mul:0", shape=(?, 256, 480, 4), dtype=float32)
Tensor("Cast:0", shape=(?, 256, 480, 4), dtype=int8)
Tensor("Cast:0", shape=(?, 256, 480, 4), dtype=int8)


In [6]:
def read_raw_data(fname):
    xs=0; ys=0
    xe=xs+2048; ye=ys+3840
    pop=np.fromfile(fname,dtype=np.dtype('i2'))
    pop=pop[0:3864*2174]
    
    pop=np.reshape(pop,(2174,3864))
    pop=np.rot90(np.rot90(pop))  #For 1, 2, comment out if running 3
    
    pop=pop[xs:xe,ys:ye]
    #pop=np.rot90(np.rot90(pop))  #For 3, comment out if running 1, 2
    
    pop=pop/(np.power(2,14)-1.)
    
    return pop

In [7]:
def trans_JPEG(data_in, QP, j_name):
    im = Image.fromarray(data_in)
    im.save(j_name,quality=QP)
    size=os.path.getsize(j_name)
    tim_t=Image.open(j_name)
    tima_t=np.array(tim_t)

    return tima_t, size

In [8]:
def write_COMPINT_JPEG(r_name, j_name, Q_P):
    Data_t = np.zeros([1,2048,3840,1])
    Data_t[0,:,:,0] = read_raw_data(r_name)
    print("  Data_t shape= " + str(Data_t[0,:,:,0].shape) + " , max= " + str(np.max(Data_t[0,:,:,0])) 
        + " , min= " + str(np.min(Data_t[0,:,:,0])) + " , mean= " + str(np.mean(Data_t[0,:,:,0])))
    
    with tf.Session() as sess:
        comp_out=sess.run(comp_o,feed_dict={Data_in:Data_t})
    
    print("  comp_out shape= " + str(comp_out.shape) + " , type= " + str(comp_out.dtype) + " , max= " + str(np.max(comp_out)) 
        + " , min= " + str(np.min(comp_out)) + " , mean= " + str(np.mean(comp_out)))
    
    comp_u8 = comp_out + 128
    comp_u8 = comp_u8.astype(np.uint8)
    print("  comp_u8 shape= " + str(comp_u8.shape) + " , type= " + str(comp_u8.dtype) + " , max= " + str(np.max(comp_u8)) 
        + " , min= " + str(np.min(comp_u8)) + " , mean= " + str(np.mean(comp_u8)))
    
    for k in range(4):
        jname = j_name + "_C_" + str(k) + ".jpeg"
        print("\n  jname: " + jname)
            
        rtima, size = trans_JPEG(comp_u8[0,:,:,k], Q_P, jname)
        print("  rtima shape= " + str(rtima.shape) + " , type= " + str(rtima.dtype) + " , max= " + str(np.max(rtima)) 
            + " , min= " + str(np.min(rtima)) + " , mean= " + str(np.mean(rtima)))
        print("  size= " + str(size))
        print("  ratio= " + str(rtima.shape[0]*rtima.shape[1]/size))
            
        psnr =compare_psnr(rtima, comp_u8[0,:,:,k])
        ssim =compare_ssim(rtima, comp_u8[0,:,:,k], multichannel=False)
            
        print("  psnr= " + str(psnr))
        print("  ssim= " + str(ssim))

In [9]:
fileName='../Example_raw_bayer_data/1.raw'

In [10]:
jpname = './tmp_COMPINT'
Q_P = 85
write_COMPINT_JPEG(fileName, jpname, Q_P)

  Data_t shape= (2048, 3840) , max= 0.9999389611182323 , min= 0.05273759384728072 , mean= 0.27559552881423693
  comp_out shape= (1, 256, 480, 4) , type= int8 , max= 72 , min= -56 , mean= 12.194521077473958
  comp_u8 shape= (1, 256, 480, 4) , type= uint8 , max= 200 , min= 72 , mean= 140.19452107747395

  jname: ./tmp_COMPINT_C_0.jpeg
  rtima shape= (256, 480) , type= uint8 , max= 127 , min= 70 , mean= 112.30228678385417
  size= 6520
  ratio= 18.846625766871167
  psnr= 48.89673744382001
  ssim= 0.9887758285664967

  jname: ./tmp_COMPINT_C_1.jpeg
  rtima shape= (256, 480) , type= uint8 , max= 198 , min= 126 , mean= 148.89544270833332
  size= 8457
  ratio= 14.529975168499467
  psnr= 46.41770130313431
  ssim= 0.9836571818002637

  jname: ./tmp_COMPINT_C_2.jpeg
  rtima shape= (256, 480) , type= uint8 , max= 204 , min= 122 , mean= 151.47864583333333
  size= 9463
  ratio= 12.98531121208919
  psnr= 46.638131242741025
  ssim= 0.9856477061888516

  jname: ./tmp_COMPINT_C_3.jpeg
  rtima shape= (25